In [52]:
import os
import json
import pandas as pd
import traceback



In [53]:
from langchain_openai import ChatOpenAI

In [54]:
from dotenv import load_dotenv
load_dotenv()

True

In [55]:
KEY=os.getenv("api_key")

In [56]:
llm=ChatOpenAI(
               model="deepseek/deepseek-r1:free",   
               openai_api_key=KEY ,  
               openai_api_base="https://openrouter.ai/api/v1",
               temperature=0.7,          
                )

In [57]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001BA1299FD90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001BA12AF4190>, root_client=<openai.OpenAI object at 0x000001BA1299F9D0>, root_async_client=<openai.AsyncOpenAI object at 0x000001BA1299FED0>, model_name='deepseek/deepseek-r1:free', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://openrouter.ai/api/v1')

In [58]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain  
import PyPDF3

In [59]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [60]:
TEMPLATE = """
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs.

### RESPONSE_JSON
{response_json}
 "IMPORTANT: Respond with only valid JSON exactly matching RESPONSE_JSON. Do NOT add any explanation."

"""

In [61]:

quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [62]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [63]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [64]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [65]:
review_chain = LLMChain(
    llm=llm,
    prompt=quiz_evaluation_prompt,
    output_key="review",
    verbose=True
)

In [66]:

#from langchain.schema.runnable import RunnableSequence   
#generate_evaluate_chain = quiz_chain | review_chain

generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [67]:
file_path =r"C:\Users\Hp\mcqgen\data.txt"

In [68]:
file_path  

'C:\\Users\\Hp\\mcqgen\\data.txt'

In [69]:
with open(file_path, "r", encoding="utf-8") as file:
    TEXT = file.read()

In [70]:
print(TEXT)

Generative artificial intelligence (Generative AI, GenAI,[1] or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2][3][4] These models learn the underlying patterns and structures of their training data and use them to produce new data[5][6] based on the input, which often comes in the form of natural language prompts.[7][8]

Generative AI tools have become more common since the AI boom in the 2020s. This boom was made possible by improvements in transformer-based deep neural networks, particularly large language models (LLMs). Major tools include chatbots such as ChatGPT, Copilot, Gemini, Claude, Grok, and DeepSeek; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo and Sora.[9][10][11][12][13] Technology companies developing generative AI include OpenAI, xAI, Anthropic, Meta AI, Microsoft, Google, DeepSeek, and Baidu.[7][14][15]

Generative AI has

In [71]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [72]:
NUMBER=5
SUBJECT="generativeai"
TONE="simple"

In [73]:
inputs = {
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "response_json": json.dumps(RESPONSE_JSON),
}
response = generate_evaluate_chain.invoke(inputs)  
response 



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Generative artificial intelligence (Generative AI, GenAI,[1] or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2][3][4] These models learn the underlying patterns and structures of their training data and use them to produce new data[5][6] based on the input, which often comes in the form of natural language prompts.[7][8]

Generative AI tools have become more common since the AI boom in the 2020s. This boom was made possible by improvements in transformer-based deep neural networks, particularly large language models (LLMs). Major tools include chatbots such as ChatGPT, Copilot, Gemini, Claude, Grok, and DeepSeek; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo and Sora.[9][10][11][12][13] Technology companies developing generative AI i

{'text': 'Generative artificial intelligence (Generative AI, GenAI,[1] or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, or other forms of data.[2][3][4] These models learn the underlying patterns and structures of their training data and use them to produce new data[5][6] based on the input, which often comes in the form of natural language prompts.[7][8]\n\nGenerative AI tools have become more common since the AI boom in the 2020s. This boom was made possible by improvements in transformer-based deep neural networks, particularly large language models (LLMs). Major tools include chatbots such as ChatGPT, Copilot, Gemini, Claude, Grok, and DeepSeek; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo and Sora.[9][10][11][12][13] Technology companies developing generative AI include OpenAI, xAI, Anthropic, Meta AI, Microsoft, Google, DeepSeek, and Baidu.[7][14][15]\n\nGen

In [74]:
import json

raw_quiz = response.get("quiz", "").strip()

# Try parsing JSON safely
try:
    if raw_quiz.startswith("```"):
        raw_quiz = raw_quiz.strip("`")
        if raw_quiz.startswith("json"):
            raw_quiz = raw_quiz[4:].strip()
    
    quiz = json.loads(raw_quiz)
except json.JSONDecodeError as e:
    print("Failed to decode quiz JSON:", e)
    print("Raw output was:", raw_quiz)
    quiz = {}


In [75]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [f"{option}: {option_value}" for option, option_value in value["options"].items()]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})


In [76]:
quiz_table_data

[{'MCQ': 'What is Generative AI a subfield of?',
  'Choices': 'a: Machine learning | b: Artificial intelligence | c: Data science | d: Computer vision',
  'Correct': 'b'},
 {'MCQ': 'What technological advancement enabled the AI boom in the 2020s?',
  'Choices': 'a: Markov chains | b: Transformer-based deep neural networks | c: Symbolic AI methods | d: State space search',
  'Correct': 'b'},
 {'MCQ': 'Which of these is an ethical concern associated with Generative AI?',
  'Choices': 'a: Creating renewable energy | b: Enhancing job opportunities | c: Deepfakes and fake news | d: Improving data privacy',
  'Correct': 'c'},
 {'MCQ': 'What is considered the first example of algorithmically generated media?',
  'Choices': "a: AARON's paintings | b: Markov chain | c: Deep neural networks | d: Text-to-image models",
  'Correct': 'b'},
 {'MCQ': 'When did generative neural networks become prominent?',
  'Choices': 'a: 1980s | b: 1990s | c: 2014–2019 | d: 2020s',
  'Correct': 'c'}]

In [78]:
quiz=pd.DataFrame(quiz_table_data)


In [80]:
quiz.to_csv("Generativeai.csv",index=False)

In [77]:
from datetime import datetime
f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

'08_22_2025_19_53_46.log'